In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Temperature Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 10


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
1,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
2,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
3,TANGARA_2B42,d29e6pg,D29ESP32DED2B42,3.4270477294921875 -76.54792785644531,3.427048,-76.547928
4,TANGARA_2E9A,d29ed62,D29ESP32DED2E9A,3.4407806396484375 -76.54106140136719,3.440781,-76.541061


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-11-09T00:00:00-05:00 -> 1699506000000, Until: 2023-11-10T23:59:59-05:00 -> 1699678799000
Group by Time: 30s


In [4]:
# Temperature Data Frame Sensors
df_temp_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'tmp', GROUP_BY_TIME)
df_temp_raw.head()
#df_temp_raw.shape

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,
2023-11-09 00:00:00-05:00,NaN,NaN,NaN,28.05,NaN,NaN,NaN,NaN,29.86,NaN
2023-11-09 00:00:30-05:00,29.63,27.26,NaN,28.04,30.16,29.27,NaN,29.7,29.84,29.36
2023-11-09 00:01:00-05:00,29.62,27.29,29.12,28.05,30.14,29.26,NaN,29.7,29.83,29.36
2023-11-09 00:01:30-05:00,29.57,27.24,29.13,28.02,30.11,29.23,NaN,29.7,29.82,29.38
2023-11-09 00:02:00-05:00,29.59,27.22,29.10,28.04,30.09,29.22,NaN,29.6,29.82,29.41


## Descriptive Statistics

In [5]:
# Describe Data
df_temp_raw.describe()

,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5609.000000,5639.000000,3836.000000,4181.000000,5666.000000,5683.000000,3226.000000,5506.000000,5723.000000,5522.000000
mean,30.826689,30.270764,30.300318,29.335044,32.799261,31.924418,31.135524,31.370287,31.604080,31.581588
std,2.354317,3.917487,3.223426,1.570660,3.016719,3.721336,3.970894,3.705822,2.383415,3.007476
min,27.050000,25.050000,27.050000,26.670000,29.230000,26.770000,26.820000,26.200000,28.090000,27.250000
25%,28.600000,26.670000,27.660000,28.250000,30.580000,28.490000,27.770000,28.100000,29.150000,28.670000
50%,30.580000,29.040000,29.060000,28.930000,31.390000,31.060000,29.420000,30.200000,31.370000,30.915000
75%,32.850000,33.840000,32.630000,30.610000,35.460000,35.460000,33.350000,35.175000,33.470000,34.560000
max,35.710000,38.090000,37.460000,32.790000,40.630000,39.800000,40.510000,38.900000,35.950000,37.190000


## Missing Data

In [6]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_temp_raw.columns:
    df_temp_sensor = df_temp_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_temp_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_14D6, Data: 67%, Missing: 33%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 73%, Missing: 27%, To be checked
Tangara Sensor: TANGARA_307A, Data: 56%, Missing: 44%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_temp_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_temp_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_temp_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_temp_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Temperature Data Frame Sensors into CSV file
df_to_csv(df_temp_raw, "temp_raw.csv")